In [3]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import altair as alt
import geopandas as gpd
import pandas as pd
import toolz

import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox
from IPython.display import display, clear_output

def custom(data):
    return toolz.curried.pipe(data, alt.to_json(filename='altdata/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('custom', custom)
alt.data_transformers.enable('custom')


DataTransformerRegistry.enable('custom')

In [66]:
# Load the shapefile
shp_data = gpd.read_file('departements-version-simplifiee.geojson')
shp_data = shp_data[shp_data['nom'] != 'NaN']
shp_data = shp_data[shp_data['nom'] != 'Haute-Corse']
shp_data = shp_data[shp_data['nom'] != 'Corse-du-Sud']

shp_data['centroid_x'] = shp_data.geometry.centroid.x
shp_data['centroid_y'] = shp_data.geometry.centroid.y

names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)
names.drop(names[names.dpt == 'Nan'].index, inplace=True)

# Select most given names
ids = names.groupby(['dpt','annais','sexe'])['nombre'].idxmax()
most_given_names = names.loc[ids]

# Aggregate the names by year
most_given_names_aggregated = most_given_names.groupby(['annais', 'preusuel', 'sexe']).agg({'nombre': 'sum'}).reset_index()
# Rank the names globally by year
most_given_names_aggregated['rank'] = most_given_names_aggregated.groupby(['annais', 'sexe'])['nombre'].rank(method='max', ascending=False).astype(int)

most_given_names = most_given_names.merge(most_given_names_aggregated[['annais', 'preusuel', 'rank']], on=['annais', 'preusuel'], how='left')

# Merge with geoshape
global_gdf = shp_data.merge(most_given_names, how='right', left_on='code', right_on='dpt')

print(most_given_names_aggregated[most_given_names_aggregated['annais'] == '1998'].sort_values('rank'))

     annais   preusuel  sexe  nombre  rank
1343   1998        LÉA     2    3894     1
1354   1998     THOMAS     1    5042     1
1351   1998    QUENTIN     1    1477     2
1344   1998      MANON     2    2499     2
1334   1998    CAMILLE     2     947     3
1342   1998      LUCAS     1     984     3
1345   1998      MARIE     2     834     4
1331   1998  ALEXANDRE     1     916     4
1349   1998     OCÉANE     2     575     5
1348   1998    NICOLAS     1     441     5
1337   1998       EMMA     2     356     6
1347   1998     MAXIME     1     297     6
1353   1998      SARAH     2     177     7
1335   1998    CLÉMENT     1     244     7
1341   1998      LAURA     2      83     8
1333   1998    ANTOINE     1     228     8
1350   1998    PAULINE     2      48     9
1336   1998      DYLAN     1     218     9
1340   1998       JEAN     1     216    10
1346   1998     MARINE     2      31    10
1355   1998   VALENTIN     1      56    11
1352   1998   SANDRINE     2      18    11
1339   1998

In [81]:
year = 1989
subset = global_gdf[global_gdf['annais'] == str(year)]

# Separate subsets for boys and girls
subset_girls = subset[subset['sexe'] == 2]
subset_boys = subset[subset['sexe'] == 1]


# Merge boys and girls data to create tooltip information
tooltip_data = subset_girls[['code', 'preusuel', 'rank']].merge(
    subset_boys[['code', 'preusuel', 'rank']],
    on='code',
    suffixes=('_girl', '_boy')
)

# Merge tooltip data back with shape data
merged_data = shp_data.merge(tooltip_data, how='left', on='code')

# Create Altair chart
chart = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
    tooltip=[alt.Tooltip('nom:N', title='Region'),
             alt.Tooltip('rank_girl:O', title='Rank girl'),
             alt.Tooltip('rank_boy:O', title='Rank boy'),
             alt.Tooltip('preusuel_girl:N', title='Most Given Name (Girl)'),
             alt.Tooltip('preusuel_boy:N', title='Most Given Name (Boy)')],
    color=alt.value('lightgrey')
).properties(
    width=800,
    height=600
)

seuil_minimal = 0.25

# Filtrer les départements par taille
filtered_data = merged_data[merged_data['geometry'].area > seuil_minimal]

# Labels pour les prénoms des filles (filtre appliqué)
text_girl = alt.Chart(filtered_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=10,
    color='red'
).encode(
    longitude='centroid_x:Q',
    latitude='centroid_y:Q',
    text='preusuel_girl:N'
)

# Labels pour les prénoms des garçons (filtre appliqué)
text_boy = alt.Chart(filtered_data).mark_text(
    align='center',
    baseline='middle',
    dy=10,  # Décalage vertical
    fontSize=10,
    color='blue'
).encode(
    longitude='centroid_x:Q',
    latitude='centroid_y:Q',
    text='preusuel_boy:N'
)

# Display the chart
chart + text_girl + text_boy

alt.LayerChart(...)

In [90]:
def createMaps(year):

    subset = global_gdf[global_gdf['annais'] == str(year)]

    # Separate subsets for boys and girls
    subset_girls = subset[subset['sexe'] == 2]
    subset_boys = subset[subset['sexe'] == 1]

    # Merge boys and girls data to create tooltip information
    tooltip_data = subset_girls[['code', 'preusuel', 'rank']].merge(
        subset_boys[['code', 'preusuel', 'rank']],
        on='code',
        suffixes=('_girl', '_boy')
    )

    # Merge tooltip data back with shape data
    merged_data = shp_data.merge(tooltip_data, how='left', on='code')

    chartG = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
        tooltip=[alt.Tooltip('nom:N', title='Region'),
                alt.Tooltip('preusuel_girl:N', title='Most Given Name (Girl)')],
        color=alt.Color('rank_girl:O', scale=alt.Scale(scheme='rainbow', reverse=True))
    ).properties(
        width=800,
        height=600
    )

    chartB = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
        tooltip=[alt.Tooltip('nom:N', title='Region'),
                alt.Tooltip('preusuel_boy:N', title='Most Given Name (Boy)')],
        color=alt.Color('rank_boy:O', scale=alt.Scale(scheme='rainbow', reverse=True))
    ).properties(
        width=800,
        height=600
    )

    seuil_minimal = 0.25

    # Filtrer les départements par taille
    filtered_data = merged_data[merged_data['geometry'].area > seuil_minimal]

    # Labels pour les prénoms des filles (filtre appliqué)
    text_girl = alt.Chart(filtered_data).mark_text(
        align='center',
        baseline='middle',
        fontSize=10,
        color='black'
    ).encode(
        longitude='centroid_x:Q',
        latitude='centroid_y:Q',
        text='preusuel_girl:N'
    )

    text_boy = alt.Chart(filtered_data).mark_text(
        align='center',
        baseline='middle',
        dy=10,  # Décalage vertical
        fontSize=10,
        color='black'
    ).encode(
        longitude='centroid_x:Q',
        latitude='centroid_y:Q',
        text='preusuel_boy:N'
    )

    # Display the chart
    return (chartG + text_girl) & (chartB + text_boy)

In [91]:
year_slider = widgets.IntSlider(value=2010, min=1900, max=2020, step=1, description='Year')

# Conteneur pour le graphique
output = widgets.Output()


def update_chart(year):
    with output:
        clear_output(wait=True)
        chart = createMaps(year)
        display(chart)

# Configurer l'interaction
interactive_plot = interactive(update_chart, year=year_slider)

# Layout pour éviter la duplication des widgets
layout = VBox([HBox([year_slider]), output])

# Affichage
display(layout)

# Initialiser le graphique avec les valeurs par défaut
update_chart(year_slider.value)